# Import Packages

In [1]:
import sys
import os
import shutil
import librosa
import glob
import tqdm
import json
import ast
import random

from librosa import display as ld

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras import models, layers
from tensorflow.keras import backend as K

!pip install git+https://github.com/AI-Guru/ngdlm.git
tf.compat.v1.disable_eager_execution()

from sklearn.preprocessing import StandardScaler
from sklearn.externals.joblib import load, dump
from sklearn import metrics
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from IPython import display as ipd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import joblib
from configparser import ConfigParser

  Cloning https://github.com/AI-Guru/ngdlm.git to /tmp/pip-req-build-9om7ay7v
  Running command git clone -q https://github.com/AI-Guru/ngdlm.git /tmp/pip-req-build-9om7ay7v
  Created wheel for ngdlm: filename=ngdlm-0.0.3-py2.py3-none-any.whl size=32027 sha256=2c87a87b36bf90da279551b874a51b9431e4cae7f04a9c1de9c2c6d926c0bc0a
  Stored in directory: /tmp/pip-ephem-wheel-cache-u17w0e7z/wheels/93/06/27/e156acb49f475c364c3c9fa4ad4ab7bfa38808bff5bf9c4647
Successfully built ngdlm


/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


# Mount Google Drive
#### 1. First mount

In [2]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/MyDrive/sound-of-failure

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/MyDrive/sound-of-failure


# Import Own Modules

In [3]:
sys.path += ['src/01_data_processing', 'src/02_modelling', 'src/00_utils']

import spectrogram as spec
import train_test_split as splt
import train_model_autoencoder as train
import naming

# Global constants

### First run the scripts for building the config files.

Some of the parameters can be passed through the command line.
At present the command line parameters are the following :

1. conf_base.py : Script for building the base config file (saves in sound-of-failure/conf/conf_base.ini)
        -raw : Pass the location of the raw data dir


2. conf_convAE.py : Script for building the config file holding the parameters for Mel Spectrogram and Convolutional Autoencoder (saves in sound-of-failure/conf/make_conf_convAE.ini). Convolutional Autoencoders can be either Autoencoders or Variational Autoencoders.
        -ae : AE or VAE
        -mel : No. of mels
        -fft : No. of FFT bands
        -hop : Hop length for the sliding window while calculating FFT
        -dim : Time dimension of one spectrogram block after chunking the whole spectrogram
        -s : Step for the sliding window for creating chunks from one spectrogram


To check the description of the command line parameters one could activate the '-h' flag. For example :

In [ ]:
%run /gdrive/MyDrive/sound-of-failure/src/00_utils/make_conf_convAE.py -h

usage: make_conf_convAE.py [-h] [-ae] [-mel] [-fft] [-hop] [-dim] [-s]

Params for Spectrogram and Autoencoder (AE or VAE)

optional arguments:
  -h, --help            show this help message and exit
  -ae , --ae            Type of Autoencoder (AE or VAE)
  -mel , --n_mels       No. of mel bands
  -fft , --n_fft        No. of FFT bands
  -hop , --hop_length   Hop length for FFT calc
  -dim , --dim          Time dimension of Spectrogram block
  -s , --step           Sliding window step for Spectrogram chunking


#### Note the change here : While running VAE we have to specify -ae to VAE in the command line. This ensures that the right config file is created (conf/conf_VAE.ini).

In [7]:
%run /gdrive/MyDrive/sound-of-failure/src/00_utils/make_conf_base.py -raw '/gdrive/MyDrive/mimii_baseline/dataset' -prj '/gdrive/MyDrive'
%run /gdrive/MyDrive/sound-of-failure/src/00_utils/make_conf_convAE.py -prj '/gdrive/MyDrive' -ae VAE -mel 16 -fft 1024 -hop 512 -dim 32 -s 8

### Then read from the config files

There are two config files : 

1. sound-of-failure/conf/**conf_base**.ini
2. sound-of-failure/conf/**conf_VAE**.ini

##### Note the change here : In normal Autoencoders the created config file is called conf_convAE.ini. Now is is called conf_VAE.ini.

In [4]:
base_conf = train.read_config('/gdrive/MyDrive/sound-of-failure/conf/conf_base.ini')
param_conf = train.read_config('/gdrive/MyDrive/sound-of-failure/conf/conf_VAE.ini')

#### Read the parameters directly from the config files

Question : Should the reading of config files be done within notebooks or the source script should directly read from them?

In [5]:
# Directories
RAW_DATA_DIR = base_conf['directories']['raw_data_dir']
BASE_DIR = base_conf['directories']['base_dir']


# Mel spectrograms
N_MELS = param_conf.getint('melspec', 'n_mels')
N_FFT = param_conf.getint('melspec', 'n_fft')
HOP_LENGTH = param_conf.getint('melspec', 'hop_length')
POWER = param_conf.getfloat('melspec', 'power')
WINDOW = param_conf.get('melspec', 'window')


# Subsampling
DIM = param_conf.getint('melspec', 'dim')
STEP = param_conf.getint('melspec', 'step')


# Scaler and cost function
SCALER_TYPE = param_conf.get('model', 'scaler')
LOSS = param_conf.get('model', 'loss')


# Optimizer
OPTIMIZER = param_conf.get('model', 'optimizer')
EPOCHS = param_conf.getint('model', 'epochs')
BATCH_SIZE = param_conf.getint('model', 'batch_size')
VALIDATION_SPLIT = param_conf.getfloat('model', 'val_split')
SHUFFLE = param_conf.getboolean('model', 'shuffle')


# Autoencoder
INPUT_SHAPE = (param_conf.getint('melspec', 'dim'),
               param_conf.getint('melspec', 'n_mels'),
               1)

LATENT_DIM = param_conf.getint('autoencoder', 'latentdim')
NUM_NODES = json.loads(param_conf.get('autoencoder', 'num_nodes'))
NUM_KERNEL = json.loads(param_conf.get('autoencoder', 'num_kernel'))
NUM_STRIDES = param_conf.get('autoencoder', 'num_strides')
NUM_STRIDES = list(ast.literal_eval(NUM_STRIDES))


# Data
DB = param_conf.get('data', 'noise')
MACHINE_TYPE = param_conf.get('data', 'machine')
MACHINE_ID = param_conf.get('data', 'machine_id')

# Demo of new naming convention

In [11]:
MODEL_NAME = 'VAE'

print(f'Default name string: {naming.get_default_name_string(MODEL_NAME, DB, MACHINE_TYPE, MACHINE_ID)}')
print(f'New name string: {naming.get_new_name_string(MODEL_NAME, DB, MACHINE_TYPE, MACHINE_ID)}\n')

name_string = naming.get_default_name_string(MODEL_NAME, DB, MACHINE_TYPE, MACHINE_ID)

print(f'Scaler path: {naming.get_scaler_path(name_string)}')
print(f'Config path: {naming.get_conf_path(name_string)}')
print(f'Model path: {naming.get_model_path(name_string)}')

Default name string: VAE_6dB_valve_id_00
New name string: VAE_6dB_valve_id_00

Scaler path: models/VAE_6dB_valve_id_00.gz
Config path: conf/VAE_6dB_valve_id_00.ini
Model path: models/VAE_6dB_valve_id_00.hdf5


# Execute preprocessing steps
#### 1. Generate unscaled mel spectrograms (~ 5 minutes)

In [11]:
spec.make_mel_dirs(BASE_DIR, DB, MACHINE_TYPE, MACHINE_ID)

spec.make_mels(RAW_DATA_DIR, BASE_DIR, 
               DB, MACHINE_TYPE, MACHINE_ID, 
               N_MELS, N_FFT, HOP_LENGTH, POWER, WINDOW, overwrite=False)

Directory already exists: /gdrive/MyDrive/sound-of-failure/data/mel_spectrograms/6dB/valve/id_00/normal
Directory already exists: /gdrive/MyDrive/sound-of-failure/data/mel_spectrograms/6dB/valve/id_00/abnormal


  0%|          | 0/991 [00:00<?, ?it/s]

Generate normal spectrograms and save to /gdrive/MyDrive/sound-of-failure/data/mel_spectrograms/6dB/valve/id_00/normal.


  0%|          | 0/119 [00:00<?, ?it/s]

Created 0 new spectrograms, kept 991 existing spectrograms.
Generate abnormal spectrograms and save to /gdrive/MyDrive/sound-of-failure/data/mel_spectrograms/6dB/valve/id_00/abnormal.


100%|██████████| 119/119 [00:01<00:00, 77.35it/s]

Created 0 new spectrograms, kept 119 existing spectrograms.


#### 2. Separate train and test files

In [12]:
train_files, train_labels, test_files, test_labels = splt.make_train_test_split(BASE_DIR, 
                                                                                DB, MACHINE_TYPE, MACHINE_ID, 
                                                                                random_seed=1)

#### 3. Scaling

In [17]:
# Previous, still valid

#scaler = spec.create_scaler(SCALER_TYPE)
#spec.fit_scaler_to_mel_files(scaler, train_files)

In [18]:
scaler = spec.fit_and_save_scaler(SCALER_TYPE, train_files, name_string, overwrite=True)

  5%|▍         | 41/872 [00:00<00:02, 404.43it/s]

Overwriting existing scaler models/VAE_6dB_valve_id_00.gz.


100%|██████████| 872/872 [00:02<00:00, 378.58it/s]

Saving scaler to models/VAE_6dB_valve_id_00.gz.


In [19]:
scaler = spec.fit_and_save_scaler(SCALER_TYPE, train_files, name_string, overwrite=False)

Loading existing scaler models/VAE_6dB_valve_id_00.gz.
